In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
#The compose function allows for multiple transforms
#transforms.ToTensor() converts our PILImage to a tensor of shape (C x H x W) in the range [0,1]
#transforms.Normalize(mean,std) normalizes a tensor to a (mean, std) for (R, G, B)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./cifardata', train=True, download=True, transform=transform)

test_set = torchvision.datasets.CIFAR10(root='./cifardata', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
from torch.utils.data.sampler import SubsetRandomSampler

#Training
n_training_samples = 20000
train_sampler = SubsetRandomSampler(np.arange(n_training_samples, dtype=np.int64))

#Validation
n_val_samples = 5000
val_sampler = SubsetRandomSampler(np.arange(n_training_samples, n_training_samples + n_val_samples, dtype=np.int64))

#Test
n_test_samples = 5000
test_sampler = SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))

In [6]:
from torch.autograd import Variable
import torch.nn.functional as F

class CNN1(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(CNN1, self).__init__()
        #Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18 * 16 * 16, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(64, 10)
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)
        x = F.relu(self.conv1(x))
        
        #Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)
        
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 18 * 16 *16)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

In [7]:
def outputSize(in_size, kernel_size, stride, padding):

    output = int((in_size - kernel_size + 2*(padding)) / stride) + 1

    return(output)

In [8]:
def get_train_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                           sampler=train_sampler, num_workers=2)
    return(train_loader)

In [9]:
#Test and validation loaders have constant batch sizes, so we can define them directly
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, sampler=test_sampler, num_workers=2)
val_loader = torch.utils.data.DataLoader(train_set, batch_size=128, sampler=val_sampler, num_workers=2)

In [10]:
import torch.optim as optim

def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [11]:
import time

def trainNet(net, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Get training data
    train_loader = get_train_loader(batch_size)
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data[0]
            total_train_loss += loss_size.data[0]
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in val_loader:
            
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.data[0]
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [13]:
CNN = CNN1()
trainNet(CNN, batch_size=32, n_epochs=20, learning_rate=0.001)

===== HYPERPARAMETERS =====
batch_size= 32
epochs= 20
learning_rate= 0.001


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 1, 10% 	 train_loss: 2.12 took: 1.36s
Epoch 1, 20% 	 train_loss: 1.87 took: 1.23s
Epoch 1, 30% 	 train_loss: 1.72 took: 1.24s
Epoch 1, 40% 	 train_loss: 1.65 took: 1.26s
Epoch 1, 50% 	 train_loss: 1.63 took: 1.26s
Epoch 1, 60% 	 train_loss: 1.51 took: 1.27s
Epoch 1, 70% 	 train_loss: 1.51 took: 1.24s
Epoch 1, 80% 	 train_loss: 1.48 took: 1.28s
Epoch 1, 90% 	 train_loss: 1.42 took: 1.31s


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:69: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Validation loss = 1.36
Epoch 2, 10% 	 train_loss: 1.35 took: 1.74s
Epoch 2, 20% 	 train_loss: 1.33 took: 2.87s
Epoch 2, 30% 	 train_loss: 1.33 took: 2.98s
Epoch 2, 40% 	 train_loss: 1.33 took: 2.57s
Epoch 2, 50% 	 train_loss: 1.29 took: 2.25s
Epoch 2, 60% 	 train_loss: 1.32 took: 2.23s
Epoch 2, 70% 	 train_loss: 1.29 took: 2.22s
Epoch 2, 80% 	 train_loss: 1.27 took: 2.23s
Epoch 2, 90% 	 train_loss: 1.29 took: 2.20s
Validation loss = 1.24
Epoch 3, 10% 	 train_loss: 1.19 took: 2.35s
Epoch 3, 20% 	 train_loss: 1.17 took: 2.26s
Epoch 3, 30% 	 train_loss: 1.22 took: 2.25s
Epoch 3, 40% 	 train_loss: 1.18 took: 2.21s
Epoch 3, 50% 	 train_loss: 1.20 took: 2.31s
Epoch 3, 60% 	 train_loss: 1.20 took: 2.22s
Epoch 3, 70% 	 train_loss: 1.22 took: 2.27s
Epoch 3, 80% 	 train_loss: 1.16 took: 2.35s
Epoch 3, 90% 	 train_loss: 1.17 took: 2.31s
Validation loss = 1.21
Epoch 4, 10% 	 train_loss: 1.06 took: 2.35s
Epoch 4, 20% 	 train_loss: 1.09 took: 2.28s
Epoch 4, 30% 	 train_loss: 1.09 took: 2.29s
Epoch 4

# Exercise 1:
# Modify the function "trainNet" so that it will also generate a plot for training loss and validation loss for each epoch.

# Exercise 2:
# Create a new "CNN2" class with network described as :
# Input, conv 3x3 10 kernels 'same' padding, ReLu, Draopout 0.1, pool max 2x2, conv 3x3 20 kernels 'same' padding, ReLu, Dropout 0.2, pool max 2x2, flatten, FC/Dense 200, FC/Dense 100, Softmax 10.
# Train this class using the "trainNet" function and same dataset.

In [ ]:
class CNN2(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(CNN2, self).__init__()
        
    def forward(self, x):
        
        pass
    